In [1]:
def simpledecorateur(function):
    def wrapper():
        print(" Avant l'appel de la fonction")
        function()
        print(" Après l'appel de la fonction")
    return wrapper
    
def function_principale():
    print("Je suis la fonction principale")
    
decorated_function = simpledecorateur(function_principale)
decorated_function()

 Avant l'appel de la fonction
Je suis la fonction principale
 Après l'appel de la fonction


In [3]:
@simpledecorateur
def function_principalev2():
    print("Je suis la fonction principale")

function_principalev2()

Je suis la fonction principale


In [4]:
@simpledecorateur
def test(argument):
    print(argument)


In [5]:
test("Est ce que cela marche ?")

TypeError: simpledecorateur.<locals>.wrapper() takes 0 positional arguments but 1 was given

In [6]:
def simpledecorateur_with_args_kwargs(function):
    def wrapper(*args, **kwargs):
        print(" Avant l'appel de la fonction")
        function(*args, **kwargs)
        print(" Après l'appel de la fonction")
    return wrapper

@simpledecorateur_with_args_kwargs
def test2(argument):
    print(argument)

test2("3O")

 Avant l'appel de la fonction
3O
 Après l'appel de la fonction


In [9]:
class DecoratorAsClass:
    def __init__(self, function):
        self.function = function
        
    def __call__(self, *args, **kwargs):
        print(" Je suis dans la classse, avant l'appel")
        result = self.function(*args, **kwargs)
        print(" Je suis dans la classse, après l'appel")
        return result

@DecoratorAsClass
def test3(argument):
    print(argument)

test3("Je suis test 3")

 Je suis dans la classse, avant l'appel
Je suis test 3
 Je suis dans la classse, après l'appel


In [11]:
def repeat_decorator(function):
    def wrapper(*args, **kwargs):
        result = None
        for i in range(0,3):
            result = function(*args, **kwargs)
        return result
    return wrapper

@repeat_decorator
def test5():
    print(" J'ai été appelée")

test5()
    

 J'ai été appelée
 J'ai été appelée
 J'ai été appelée


In [13]:
def repeat(number):
    def repeat_decorator(function):
        def wrapper(*args, **kwargs):
            result = None
            for i in range(0,number):
                result = function(*args, **kwargs)
            return result
        return wrapper
    return repeat_decorator

@repeat(3)
def test6():
    print("J'ai été appelée")

test6()
    

J'ai été appelée
J'ai été appelée
J'ai été appelée


In [16]:
def repeat2(number = 3):
    def repeat_decorator(function):
        def wrapper(*args, **kwargs):
            result = None
            for i in range(0,number):
                result = function(*args, **kwargs)
            return result
        return wrapper
    return repeat_decorator

@repeat2
def test7():
    print("J'ai été appelée")

test7()

TypeError: repeat2.<locals>.repeat_decorator() missing 1 required positional argument: 'function'

In [21]:
def dummy_decorator(function):
    def wrapper(*args, **kwargs):
        """Internal wrapper documentation"""
        function(*args, **kwargs)
    return wrapper

@dummy_decorator
def test8():
    """ DOCUMENTATION SUPER IMPORTANTE """
    print(" test 8 ")

print(test8.__doc__)

Internal wrapper documentation


In [23]:
from functools import wraps

def repeat_decorator(function):
    @wraps(function)
    def wrapper(*args, **kwargs):
        """Internal wrapper documentation"""
        function(*args, **kwargs)
    return wrapper

@repeat_decorator
def test9():
    """ DOCUMENTATION SUPER IMPORTANTE """
    print(" test 9 ")

print(test9.__doc__)

 DOCUMENTATION SUPER IMPORTANTE 


In [25]:
print(test9.__dict__)
def test10():
    """ DOCUMENTATION SUPER IMPORTANTE """
    print(" test 9 ")
print(test10.__dict__)

{'__wrapped__': <function test9 at 0x00000239B1DFAFC0>}
{}


In [37]:
cache = {}
import pickle
import hashlib
import time 

def compute_key(function, args, kwargs):
    key = pickle.dumps((function.__name__, args, kwargs))
    index = hashlib.sha1(key).hexdigest()
    print(f"{function.__name__}, {args}, {kwargs}, donne une clé de : {index}")
    return index

def memorise(function):
    def _merosise(*args, **kwargs):
        key = compute_key(function, *args, **kwargs)
        if (key in cache):
            print("Resultat deja cache")
            return cache[key]
        print("Resultat pas cache")
        result = function(*args, **kwargs)
        cache[key] = result
        return result
    return _merosise
    
@memorise
def fonction_lourde(a, b):
    time.sleep(2)
    return a + b
    
print(cache)

print(fonction_lourde(3, 5))

print(cache)

{}
fonction_lourde, 3, 5, donne une clé de : f8a9302b625fdb422e3a8b6b549f1bc8498c069c
Resultat pas cache
8
{'f8a9302b625fdb422e3a8b6b549f1bc8498c069c': 8}


In [38]:
print(fonction_lourde(3, 5))
print(cache)

fonction_lourde, 3, 5, donne une clé de : f8a9302b625fdb422e3a8b6b549f1bc8498c069c
Resultat deja cache
8
{'f8a9302b625fdb422e3a8b6b549f1bc8498c069c': 8}


In [39]:
print(fonction_lourde(3, 6))
print(cache)

fonction_lourde, 3, 6, donne une clé de : 5a0faf89087e7c278f0618a2cc6d926b33c60e40
Resultat pas cache
9
{'f8a9302b625fdb422e3a8b6b549f1bc8498c069c': 8, '5a0faf89087e7c278f0618a2cc6d926b33c60e40': 9}


In [40]:
print(fonction_lourde(5, 3))
print(cache)

fonction_lourde, 5, 3, donne une clé de : 9ab0b555f8cdd084d06c2d9057e8309ae69f2c93
Resultat pas cache
8
{'f8a9302b625fdb422e3a8b6b549f1bc8498c069c': 8, '5a0faf89087e7c278f0618a2cc6d926b33c60e40': 9, '9ab0b555f8cdd084d06c2d9057e8309ae69f2c93': 8}


In [43]:
cache = {}

def is_obsolete(entry, duration):
    return (time.time() - entry) > duration
    
def memorise_obsolete(time_to_be_obsolete):
    def memorise(function):
        def _merosise(*args, **kwargs):
            key = compute_key(function, *args, **kwargs)
            if (key in cache and not is_obsolete(cache[key]["time"], time_to_be_obsolete)):
                print("Resultat deja cache")
                return cache[key]["value"]
            print("Resultat pas cache")
            result = function(*args, **kwargs)
            cache[key] = {"value" : result, "time":time.time()}
            return result
        return _merosise
    return memorise

@memorise_obsolete(5)
def fonction_lourde(a, b):
    time.sleep(2)
    return a + b
    
print(cache)

print(fonction_lourde(3, 5))
print(fonction_lourde(3, 5))
print(cache)

{}
fonction_lourde, 3, 5, donne une clé de : f8a9302b625fdb422e3a8b6b549f1bc8498c069c
Resultat pas cache
8
fonction_lourde, 3, 5, donne une clé de : f8a9302b625fdb422e3a8b6b549f1bc8498c069c
Resultat deja cache
8
{'f8a9302b625fdb422e3a8b6b549f1bc8498c069c': {'value': 8, 'time': 1710165466.8545485}}


In [44]:
print(fonction_lourde(3, 5))
print(cache)

fonction_lourde, 3, 5, donne une clé de : f8a9302b625fdb422e3a8b6b549f1bc8498c069c
Resultat pas cache
8
{'f8a9302b625fdb422e3a8b6b549f1bc8498c069c': {'value': 8, 'time': 1710165492.8998284}}


In [45]:
print(fonction_lourde(4, 5))
print(cache)

fonction_lourde, 4, 5, donne une clé de : 1276efa74aa9f7bb8b64e0f287053b678b386807
Resultat pas cache
9
{'f8a9302b625fdb422e3a8b6b549f1bc8498c069c': {'value': 8, 'time': 1710165492.8998284}, '1276efa74aa9f7bb8b64e0f287053b678b386807': {'value': 9, 'time': 1710165510.9061773}}


In [ ]:
import threading import RLock
lock = RLock()

def synchronized(function):
    def _synchronized(*args, **kwargs):
        lock.acquire()
        try :
            return function(*args, **kwaergs)
        finally:
            lock.release()
    return _synchronized

@synchronized
def thread_safe():
    print("Je sui à l'intérieur")

thread_safe(